In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('train.csv',index_col = 'PassengerId')
test = pd.read_csv('test.csv',index_col = 'PassengerId')
data = pd.concat([train,test])
data.head()

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket
PassengerId,,,,,,,,,,,
1,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,0.0,A/5 21171
2,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,1.0,PC 17599
3,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,1.0,STON/O2. 3101282
4,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,1.0,113803
5,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,0.0,373450


In [ ]:
def title_mapper(df):   
    Influence = ['Don', 'Col', 'Major', 'Capt', 'Sir', 'Mme', 'Lady', 'Countess'] ## Score = 3 
    Adult = ['Mr', 'Mrs', 'Miss', 'MS', 'Mlle', 'Dr', 'Rev','Ms'] ## Score = 2
    Child = ['Jonkheer','Master'] ## Score = 1
    mapper = {}
    if len(mapper) == 0:
        for i in Influence:
            mapper[i] = 3
        for i in Adult:
            mapper[i] = 2
        for i in Child:
            mapper[i] = 1
        df['Title_score'] = df.Title.map(mapper)
    df.Title_score[df.Title_score.isna()] = 2

In [ ]:
def Age_by_title(df):
    
    Miss_Age = int(  df.loc[(df.Age.notnull()) & (df.Title == 'Miss'),'Age'].mean()   )
    Mr_Age = int(    df.loc[(df.Age.notnull()) & (df.Title == 'Mr'),'Age'].mean()     )
    Mrs_Age = int(   df.loc[(df.Age.notnull()) & (df.Title == 'Mrs'),'Age'].mean()    )
    Master_Age = int(df.loc[(df.Age.notnull()) & (df.Title == 'Master'),'Age'].mean() )

    df.loc[(df.Age.isna()) & (df.Title == 'Miss'),'Age'] = Miss_Age
    df.loc[(df.Age.isna()) & (df.Title == 'Mr'),'Age'] = Mr_Age
    df.loc[(df.Age.isna()) & (df.Title == 'Mrs'),'Age'] = Mrs_Age
    df.loc[(df.Age.isna()) & (df.Title == 'Master'),'Age'] = Master_Age
    
    df.loc[df.Age.isna(),'Age'] = int(df.Age.mean())
    
    return 1 if df.Age.isna().sum() == 0 else 0

In [ ]:
data['Family_size'] = data['SibSp'] + data['Parch'] + 1
data['Title'] = [passenger[1].replace('.','').strip().split(' ')[0] for passenger in data.Name.str.split(',')]
Age_by_title(data)
if data.Sex.dtype != np.int64 :
    data.Sex = data.Sex.map({'male':1,'female':0})
title_mapper(data)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
class Familia():
    def __init__(self,sur):
        self.sur = sur
        
def Nodes_size_agg(dtf):
    nodes = []
    for index_row in dtf.index:
        nodes = nodes + int(round(dtf.iloc[index_row].Count)) * [dtf.iloc[index_row].Size]
    return nodes

def Ticket_to_number(df):
    df['Ticket_number'] = [ticket[-1] for ticket in df.Ticket.str.split(' ')]
    def int_mapper(x):
        return any(c.isalpha() for c in x)
    df.Ticket_number[df.Ticket_number.map(int_mapper)] = 0
    df['Ticket_number'] = df.Ticket_number.astype(np.int32,inplace = True)
    

def loc_family(df):
    families = list(set(df.Surname))
    family_values = []
    for family in families:
        
        familia = Familia(family)
        
        ## 1º Family_size
        Family_df = pd.DataFrame(df[df.Surname == family].Family_size.value_counts()).reset_index().rename(columns = {'index':'Size','Family_size':'N'})
        Family_df['Count'] = Family_df.N / Family_df.Size
        
        Total_Size = Family_df.Count.sum()
        
        familia.size = Nodes_size_agg(Family_df)

        ## 2º Fare       
        Total_Fare = len(df[df.Surname == family].Fare.unique())
        
        familia.fare = df[df.Surname == family].Fare.unique()
        
        ## 3º Ticket
        from collections import Counter

        tickets = list(df[data.Surname == family].Ticket_number)
        result = []
        for ticket in tickets:
            prov = []
            for index in range(0,len(tickets)):
                if tickets.index(ticket) != index:
                    if abs(ticket - tickets[index]) < 10:
                            prov.append(1)

                    else:
                            prov.append(0)
            #print(prov) << UNCOMMENT TO SEE THE MATRIX OF EACH VALUE RELATED TO OTHERS
            result.append(sum(prov)+1)
            
        Counts = Counter(result)
        Total_ticket = sum([Counts[key]/key for key in Counts.keys()])
        
        familia.ticket = list(set(tickets))
        familia.total = round((Total_Size+Total_Fare+Total_ticket)/3)
        if family == 'Davies':
            print('------')
            print()
            print('-------------------------------')
            print()
        print(f'{familia.sur},Size : {familia.size},Fare : {familia.fare},Ticket : {familia.ticket},Total :{familia.total}')
        
        nb = []
        #if familia.total > 1:
        #    for n in range(0,familia.total):
        #        df[df.Family_size == familia.size[n]]
                if len(x) == Total.size:
                    nb.append(1)
                    break
        
        
        
        
        
        
        
        ## TOTAL

        family_values.append(round((Total_Size+Total_Fare+Total_ticket)/3))
        #all_family = pd.DataFrame([all_fam,family_values])
        
    all_family = pd.DataFrame(families,family_values).reset_index().rename(columns = {'index' : 'N_of_families',0:'Family'})
    #print(all_family)
    #return all_family  # << EXCLUDE THIS AFTERWARDS
    
    
    
    
    
    
    
    
data['Surname'] = [surname[0] for surname in data.Name.str.split(',')]    
    
Ticket_to_number(data)
Fams = loc_family(data)    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Cumings,Size : [2.0],Fare : [71.2833],Ticket : [17599],Total :1.0
Levy,Size : [1.0],Fare : [12.875],Ticket : [2163],Total :1.0
Kelly,Size : [1.0, 1.0, 1.0, 1.0, 1.0],Fare : [ 7.75    8.05   13.5     7.8292],Ticket : [14312, 363592, 223596, 9234, 330911],Total :5.0
Cleaver,Size : [1.0],Fare : [151.55],Ticket : [113781],Total :1.0
Dean,Size : [4.0],Fare : [20.575],Ticket : [2315],Total :1.0
Palsson,Size : [5.0],Fare : [21.075],Ticket : [349909],Total :1.0
Frolicher-Stehli,Size : [3.0],Fare : [79.2],Ticket : [13567],Total :1.0
Lithman,Size : [1.0],Fare : [7.55],Ticket : [251],Total :1.0
Colbert,Size : [1.0],Fare : [7.25],Ticket : [371109],Total :1.0
Rosblom,Size : [3.0],Fare : [20.2125],Ticket : [370129],Total :1.0
Peacock,Size : [3.0],Fare : [13.775],Ticket : [3101315],Total :1.0
Schmidt,Size : [1.0],Fare : [13.],Ticket : [248659],Total :1.0
Karnes,Size : [1.0],Fare : [21.],Ticket : [13534],Total :1.0
Brandeis,Size : [1.0],Fare : [50.4958],Ticket : [17591],Total :1.0
Henry,Size : [1.0],F

Petersen,Size : [1.0],Fare : [8.05],Ticket : [342441],Total :1.0
Vestrom,Size : [1.0],Fare : [7.8542],Ticket : [350406],Total :1.0
Bjornstrom-Steffansson,Size : [1.0],Fare : [26.55],Ticket : [110564],Total :1.0
Pavlovic,Size : [1.0],Fare : [7.8958],Ticket : [349242],Total :1.0
Mitkoff,Size : [1.0],Fare : [7.8958],Ticket : [349221],Total :1.0
Bjorklund,Size : [1.0],Fare : [7.75],Ticket : [347090],Total :1.0
Coleridge,Size : [1.0],Fare : [10.5],Ticket : [14263],Total :1.0
Bowenur,Size : [1.0],Fare : [13.],Ticket : [211535],Total :1.0
de Mulder,Size : [1.0],Fare : [9.5],Ticket : [345774],Total :1.0
Sjoblom,Size : [1.0],Fare : [7.4958],Ticket : [3101265],Total :1.0
Zakarian,Size : [1.0, 1.0],Fare : [7.225],Ticket : [2656, 2670],Total :2.0
Harmer,Size : [1.0],Fare : [7.25],Ticket : [374887],Total :1.0
Madsen,Size : [1.0],Fare : [7.1417],Ticket : [17369],Total :1.0
Robert,Size : [],Fare : [211.3375],Ticket : [24160],Total :1.0
Shutes,Size : [1.0],Fare : [153.4625],Ticket : [17582],Total :1.0

Turkula,Size : [1.0],Fare : [9.5875],Ticket : [4134],Total :1.0
Doharr,Size : [1.0],Fare : [7.2292],Ticket : [2686],Total :1.0
Slocovski,Size : [1.0],Fare : [8.05],Ticket : [392086],Total :1.0
Murdlin,Size : [1.0],Fare : [8.05],Ticket : [3235],Total :1.0
Theobald,Size : [1.0],Fare : [8.05],Ticket : [363294],Total :1.0
Duran y More,Size : [2.0],Fare : [13.8583],Ticket : [2148, 2149],Total :1.0
Sirayanian,Size : [1.0],Fare : [7.2292],Ticket : [2669],Total :1.0
Hendekovic,Size : [1.0],Fare : [7.8958],Ticket : [349243],Total :1.0
Collett,Size : [1.0],Fare : [10.5],Ticket : [28034],Total :1.0
Willard,Size : [1.0],Fare : [26.55],Ticket : [113795],Total :1.0
Johansson,Size : [1.0, 1.0, 1.0, 1.0],Fare : [8.6542 7.7958 7.775  7.8542],Ticket : [350052, 347467, 7540, 347063],Total :4.0
Persson,Size : [],Fare : [7.775],Ticket : [347083],Total :1.0
Honkanen,Size : [1.0],Fare : [7.925],Ticket : [3101283],Total :1.0
Pengelly,Size : [1.0],Fare : [10.5],Ticket : [28665],Total :1.0
Assaf,Size : [1.0],Fa

Strom,Size : [],Fare : [10.4625],Ticket : [347054],Total :1.0
Woolner,Size : [1.0],Fare : [35.5],Ticket : [19947],Total :1.0
Brady,Size : [1.0],Fare : [30.5],Ticket : [113054],Total :1.0
Gronnestad,Size : [1.0],Fare : [8.3625],Ticket : [8471],Total :1.0
Nysten,Size : [1.0],Fare : [7.75],Ticket : [347081],Total :1.0
Pernot,Size : [1.0],Fare : [15.05],Ticket : [2131],Total :1.0
Holthen,Size : [1.0],Fare : [22.525],Ticket : [4001],Total :1.0
Zimmerman,Size : [1.0],Fare : [7.875],Ticket : [315082],Total :1.0
Backstrom,Size : [],Fare : [15.85],Ticket : [3101278],Total :1.0
Faunthorpe,Size : [2.0],Fare : [26.],Ticket : [2926],Total :1.0
Petranec,Size : [1.0],Fare : [7.8958],Ticket : [349245],Total :1.0
Staneff,Size : [1.0],Fare : [7.8958],Ticket : [349208],Total :1.0
Newell,Size : [2.0],Fare : [113.275],Ticket : [35273],Total :1.0
Tenglin,Size : [1.0],Fare : [7.7958],Ticket : [350033],Total :1.0
Fleming,Size : [1.0, 1.0],Fare : [110.8833   7.75  ],Ticket : [364859, 17421],Total :2.0
Gheorghe

Abbott,Size : [3.0],Fare : [20.25],Ticket : [2673],Total :1.0
Heininen,Size : [1.0],Fare : [7.925],Ticket : [3101290],Total :1.0
Stahelin-Maeglin,Size : [1.0],Fare : [30.5],Ticket : [13214],Total :1.0
Padro y Manent,Size : [1.0],Fare : [13.8625],Ticket : [2146],Total :1.0
Perkin,Size : [1.0],Fare : [7.25],Ticket : [21174],Total :1.0
Chevre,Size : [1.0],Fare : [29.7],Ticket : [17594],Total :1.0
O'Connell,Size : [1.0],Fare : [7.7333],Ticket : [334912],Total :1.0
Caram,Size : [2.0],Fare : [14.4583],Ticket : [2689],Total :1.0
Kilgannon,Size : [1.0],Fare : [7.7375],Ticket : [36865],Total :1.0
Fry,Size : [1.0],Fare : [0.],Ticket : [112058],Total :1.0
Strandberg,Size : [1.0],Fare : [9.8375],Ticket : [7553],Total :1.0
Baccos,Size : [1.0],Fare : [7.225],Ticket : [2679],Total :1.0
Myhrman,Size : [1.0],Fare : [7.75],Ticket : [347078],Total :1.0
Tomlin,Size : [1.0],Fare : [8.05],Ticket : [364499],Total :1.0
Gilbert,Size : [1.0],Fare : [10.5],Ticket : [30769],Total :1.0
Oxenham,Size : [1.0],Fare : 

McNeill,Size : [1.0],Fare : [7.75],Ticket : [370368],Total :1.0
Ali,Size : [1.0, 1.0],Fare : [7.05],Ticket : [3101312, 3101311],Total :1.0
Longley,Size : [1.0],Fare : [77.9583],Ticket : [13502],Total :1.0
Saether,Size : [1.0],Fare : [7.25],Ticket : [3101262],Total :1.0
Myles,Size : [1.0],Fare : [9.6875],Ticket : [240276],Total :1.0
Jussila,Size : [2.0, 1.0],Fare : [9.825 7.925],Ticket : [4136, 4137, 3101286],Total :2.0
Swane,Size : [1.0],Fare : [13.],Ticket : [248734],Total :1.0
Becker,Size : [4.0],Fare : [39.],Ticket : [230136],Total :1.0
Brocklebank,Size : [1.0],Fare : [8.05],Ticket : [364512],Total :1.0
Mitchell,Size : [1.0],Fare : [10.5],Ticket : [24580],Total :1.0
Lam,Size : [1.0, 1.0],Fare : [56.4958],Ticket : [1601],Total :1.0
Yasbeck,Size : [2.0],Fare : [14.4542],Ticket : [2659],Total :1.0
Carr,Size : [1.0, 1.0],Fare : [7.75],Ticket : [368364, 367231],Total :2.0
Burke,Size : [1.0],Fare : [6.75],Ticket : [365222],Total :1.0
Larsson-Rondberg,Size : [1.0],Fare : [7.775],Ticket : [

Buckley,Size : [1.0, 1.0],Fare : [7.8208 7.2833],Ticket : [330920, 329944],Total :2.0
Connolly,Size : [1.0, 1.0],Fare : [7.75   7.6292],Ticket : [330972, 370373],Total :2.0
Salomon,Size : [1.0],Fare : [26.],Ticket : [111163],Total :1.0
Shine,Size : [1.0],Fare : [7.7792],Ticket : [330968],Total :1.0
Thorne,Size : [1.0],Fare : [79.2],Ticket : [17585],Total :1.0
Widegren,Size : [1.0],Fare : [7.75],Ticket : [347064],Total :1.0
O'Leary,Size : [1.0],Fare : [7.8292],Ticket : [330919],Total :1.0
Hee,Size : [1.0],Fare : [56.4958],Ticket : [1601],Total :1.0
Plotcharsky,Size : [1.0],Fare : [7.8958],Ticket : [349227],Total :1.0
Penasco y Castellana,Size : [2.0],Fare : [108.9],Ticket : [17758],Total :1.0
Marvin,Size : [2.0],Fare : [53.1],Ticket : [113773],Total :1.0
Kirkland,Size : [1.0],Fare : [12.35],Ticket : [219533],Total :1.0
Glynn,Size : [1.0],Fare : [7.75],Ticket : [335677],Total :1.0
Drew,Size : [3.0],Fare : [32.5],Ticket : [28220],Total :1.0
Duane,Size : [1.0],Fare : [7.75],Ticket : [33643

In [ ]:
len(data[(data.Surname=='Davies')&(data.Family_size == 3)])

5

In [ ]:
data[data.Surname == 'Kelly'].Fare.unique()

array([ 7.75  ,  8.05  , 13.5   ,  7.8292])

In [ ]:
data[(data.Surname == 'Davies') & (data.Family_size == 3)]

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket,Family_size,Title,Title_score,Surname,Ticket_number
PassengerId,,,,,,,,,,,,,,,,
550,8.0,NaN,S,36.75,"Davies, Master. John Morgan Jr",1,2,1,1,1.0,C.A. 33112,3,Master,1.0,Davies,33112
566,24.0,NaN,S,24.15,"Davies, Mr. Alfred J",0,3,1,2,0.0,A/4 48871,3,Mr,2.0,Davies,48871
901,21.0,NaN,S,24.15,"Davies, Mr. John Samuel",0,3,1,2,NaN,A/4 48871,3,Mr,2.0,Davies,48871
1079,17.0,NaN,S,8.05,"Davies, Mr. Joseph",0,3,1,2,NaN,A/4 48873,3,Mr,2.0,Davies,48873
1222,48.0,NaN,S,36.75,"Davies, Mrs. John Morgan (Elizabeth Agnes Mary...",2,2,0,0,NaN,C.A. 33112,3,Mrs,2.0,Davies,33112


In [ ]:
pd.DataFrame(data[data.Surname == 'Kink'].Family_size.value_counts()).reset_index().rename(columns = {'index':'Size','Family_size':'N'})

,Size,N
0,3,2


In [ ]:
#Family_df = pd.DataFrame(data[data.Surname == 'Kink'].Fare.value_counts()).reset_index().rename(columns = {'index':'Fare','Family_size':'N'})
#Family_df['Count'] = Family_df.N / Family_df.Fare
round((len(data[data.Surname == 'Davies'].Fare.unique()) + Family_df.Count.sum())/2)

NameError: name 'Family_df' is not defined

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score

columns = ['Sex','Title_score','Family_size']

x = train.loc[:,columns]
y = train.loc[:,'Survived']

Dtc = DecisionTreeClassifier()
score = cross_val_score(Dtc,x,y,cv = 10,scoring = 'accuracy')
score.mean()

In [ ]:
Dtc.fit(x,y)

test['Family_size'] = test['SibSp'] + test['Parch'] + 1
test['Title'] = [passenger[1].replace('.','').strip().split(' ')[0] for passenger in test.Name.str.split(',')]
title_mapper(test)
Age_by_title(test)
test.Sex = test.Sex.map({'male':1,'female':0})


x_test = test.loc[:,columns]
y_pred = Dtc.predict(x_test)

## CREATING TXT FILE TO SUB

#Sub = pd.DataFrame({'PassengerId':test.PassengerId,'Survived':y_pred})
#with open('Submition10.txt','w') as file:
#    file.write(Sub.to_csv(index = False))

In [ ]:
data['Surname'] = [surname[0] for surname in data.Name.str.split(',')]

def loc_family(df):
    all_families = {}
    for family in set(df.Surname):
        ## 1º Family_size
        Family_df = pd.DataFrame(data[data.Surname == family].Family_size.value_counts()).reset_index().rename(columns = {'index':'Size','Family_size':'N'})
        Family_df['Count'] = Family_df.N / Family_df.Size

        
        ## 2º Fare
        
        
        Total = Family_df.Count.sum()
        all_families[family] = Total
        
    print(all_families)
    
    
    
loc_family(data)    